<a href="https://colab.research.google.com/github/xDevICCI/TallerDeepLearning/blob/main/VictorGodoy_XXXXXXXXX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='red'>WARNING-1: No se aceptan soluciones que usen metodologías distintas a las vistas en clases.</font>

<font color='orange'>Está permitido usar los códigos de los ejemplos vistos en clases.</font>



# 0) Cambiar el nombre del archivo

<font color='red'>WARNING-2: CAMBIA EL NOMBRE DEL ARCHIVO ipynb INCLUYENDO TU NOMBRE Y RUT</font>

Ejemplo, si tu nombres es Pedro Perez y RUT es 12.345.678-9, el nombre de este archivo debe cambiarse de "Nombre_Rut.ipynb" a "Pedro_Perez_12345678-9.ipynb"


# Defina acá el nombre de proyecto, por ejemplo: 'Clasificación de Sentimientod usando BiLSTM'

## Describa su dataset y el problema que quiere resolver:

**Ejemplo: **

'En este problema de clasificación hay 3 clases:

* Clase 0: 'bikes' - personas con bicicleta

* Clase 1: 'noperson' - imágenes de la ciudad sin peatones ni bicicletas

* Clase 2: 'person' - peatones (sin bicicleta)

La solución se hará es usando BiLSTM de la librería HuggingFace'.


# 1) Carga de datos y librerías a utilizar

(2 puntos)

Incluya aquí todas las librerías a utilizar en este código.

De una ubicación de los datos para ser descargados.

Por ejemplo: Los datos se encuentran en `https://www.dropbox.com/s/p94odo87ekacd46/bikes3.zip`




# 2) Definición de parámetros

(1 punto)

Defina los parámetros necesarios. Por ejemplo: `fpath` (path de los archivos), `BATCH_SIZE`, etc.



# 3) Pre-procesamiento y Data augmentation (si es necesario, investigue métodos para el aumento de los datos artificialente)

(2 puntos)

Defina los parámetros necesarios, haga los cambios necesarios considerando el tamaño de las imágenes, señales, etc.



# 4) Definir el modelo

(1 punto)

Defina la clase donde se decriba y declare el o los modelos a usar.



We will use the pretrained Vision Transformer feature extractor, an Adam Optimizer, and a Cross Entropy Loss function.

# 5) Entrenar el modelo

(4 puntos)

Entre el modelo definido en los pasos anteriores

# 6) Evaluación de desempeño

(2 puntos)

Calcule el accuracy en el training, validation y testing.

# 7) Evaluación aleatoria (de una imagen, señal etc.) del testing

(1 punto)

Despliegue una señal, imagen, cadena de texto, etc.,  aleatoria del testing con la clasificación proporcionada por el Transformer entrenado.